# Day 1: Logistic Regression
We will apply logistic regression to a binary classification problem. We will try out a few datasets, some randomly generated (i.e. artificial) and some real-life data.

Most packages will also have datasets alongwith. For instance the statsmodels package has a whole bunch of datasets as in http://statsmodels.sourceforge.net/devel/datasets/index.html

The next example is based on the following site that uses Fair's data on extramarital affairs, see
http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/discrete_choice_example.html

Also see http://nbviewer.jupyter.org/gist/justmarkham/6d5c061ca5aee67c4316471f8c2ae976

## Round 1

In [54]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols
import matplotlib.pyplot as plt
from patsy import dmatrices
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [55]:
from __future__ import print_function

The __future__ module is a real Python module. You will often see constructs like from __future__ import division. What do you think this line does? 

In [56]:
print(sm.datasets.fair.SOURCE)


Fair, Ray. 1978. "A Theory of Extramarital Affairs," `Journal of Political
    Economy`, February, 45-61.

The data is available at http://fairmodel.econ.yale.edu/rayfair/pdf/2011b.htm



In [57]:
print( sm.datasets.fair.NOTE)

::

    Number of observations: 6366
    Number of variables: 9
    Variable name definitions:

        rate_marriage   : How rate marriage, 1 = very poor, 2 = poor, 3 = fair,
                        4 = good, 5 = very good
        age             : Age
        yrs_married     : No. years married. Interval approximations. See
                        original paper for detailed explanation.
        children        : No. children
        religious       : How relgious, 1 = not, 2 = mildly, 3 = fairly,
                        4 = strongly
        educ            : Level of education, 9 = grade school, 12 = high
                        school, 14 = some college, 16 = college graduate,
                        17 = some graduate school, 20 = advanced degree
        occupation      : 1 = student, 2 = farming, agriculture; semi-skilled,
                        or unskilled worker; 3 = white-colloar; 4 = teacher
                        counselor social worker, nurse; artist, writers;
          

Load the dataset using Pandas. Note that this is a Pandas data structure, so we have to use Pandas' methods in order to access the data (say, print the first line of the data table). 

In [58]:
dta = sm.datasets.fair.load_pandas().data 
# Try out print(dta) 
print(dta.head(1))

   rate_marriage   age  yrs_married  children  religious  educ  occupation  \
0            3.0  32.0          9.0       3.0        3.0  17.0         2.0   

   occupation_husb   affairs  
0              5.0  0.111111  


__Sidenote__: Here, print dta.head(1) will not work. This is because we imported print as a function, see the line from __future__ import print_function, so print a will not work any more, since this is using print as a statement.

In [59]:
#print(dta['affairs']) 

Note that the affairs column shows the total amount of time spent in affairs. Let us try to classify whether a person will cheat or not. Hence, let us replace all entries where dta['affairs'] > 0 by 1 (rather by 1.0). We can get this by the Boolean expression (dta['affairs']> 0). 

We will make this into a new column, and call the new column "affair"

In [60]:
# (dta['affairs']> 0)
dta['affair'] = (dta['affairs'] > 0).astype(float)

In [61]:
print(dta.head(2))

   rate_marriage   age  yrs_married  children  religious  educ  occupation  \
0            3.0  32.0          9.0       3.0        3.0  17.0         2.0   
1            3.0  27.0         13.0       3.0        1.0  14.0         3.0   

   occupation_husb   affairs  affair  
0              5.0  0.111111     1.0  
1              4.0  3.230769     1.0  


People who use R will be familiar with the next few steps. We want to "describe" the dataset, i.e. get the most relevant statistics to understand this data. Also the fitting of the model (logit) in this case is quite similar to that in R.

In [62]:
print(dta.describe())

       rate_marriage          age  yrs_married     children    religious  \
count    6366.000000  6366.000000  6366.000000  6366.000000  6366.000000   
mean        4.109645    29.082862     9.009425     1.396874     2.426170   
std         0.961430     6.847882     7.280120     1.433471     0.878369   
min         1.000000    17.500000     0.500000     0.000000     1.000000   
25%         4.000000    22.000000     2.500000     0.000000     2.000000   
50%         4.000000    27.000000     6.000000     1.000000     2.000000   
75%         5.000000    32.000000    16.500000     2.000000     3.000000   
max         5.000000    42.000000    23.000000     5.500000     4.000000   

              educ   occupation  occupation_husb      affairs       affair  
count  6366.000000  6366.000000      6366.000000  6366.000000  6366.000000  
mean     14.209865     3.424128         3.850141     0.705374     0.322495  
std       2.178003     0.942399         1.346435     2.203374     0.467468  
min    

In [63]:
affair_mod = logit("affair ~ occupation + educ + occupation_husb"
                   "+ rate_marriage + age + yrs_married + children"
                   " + religious", dta).fit()

Optimization terminated successfully.
         Current function value: 0.545314
         Iterations 6


Comments about this model: 

Here, "affair" is the independent variable (also called the __target__ variable) whereas the other columns are the dependent variables. 

Note that occupation, occupation_husb are not really __numerical__ variables, but are __categorical__ variables. 
As another example, suppose color-of-car were an independent variable in some predictive model. How do we put a linear ordering on the color-of-car? 

Quiz question: how do we deal with __categorical__ variables?

In [64]:
print(affair_mod.summary())

                           Logit Regression Results                           
Dep. Variable:                 affair   No. Observations:                 6366
Model:                          Logit   Df Residuals:                     6357
Method:                           MLE   Df Model:                            8
Date:                Sat, 30 Jul 2016   Pseudo R-squ.:                  0.1327
Time:                        22:45:33   Log-Likelihood:                -3471.5
converged:                       True   LL-Null:                       -4002.5
                                        LLR p-value:                5.807e-224
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           3.7257      0.299     12.470      0.000         3.140     4.311
occupation          0.1602      0.034      4.717      0.000         0.094     0.227
educ               -0.0392      

This is a good place to discuss the various parameter values in the results. MLE refers to Maximum Likelihood Estimate that we will cover in detail in our indepth discussion of Logistic Regression later.

To discuss: 
- p-values, 
- Log-Likelihood, 
- Pseudo R-square.

For each, show 
- how to compute
- whether the values above match up.

In [65]:
affair_mod.pred_table()

array([[ 3882.,   431.],
       [ 1326.,   727.]])

The pred_table is also called the __confusion matrix__. Note that the sum of all the entries in this matrix equals the total number of observations (in this example, 6366). 

What do the terms p-value, log-likelihood etc. mean? Details at http://www.ats.ucla.edu/stat/stata/output/stata_logistic.htm


In [66]:
mfx = affair_mod.get_margeff()
print(mfx.summary())

        Logit Marginal Effects       
Dep. Variable:                 affair
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
occupation          0.0293      0.006      4.744      0.000         0.017     0.041
educ               -0.0072      0.003     -2.538      0.011        -0.013    -0.002
occupation_husb     0.0023      0.004      0.541      0.589        -0.006     0.010
rate_marriage      -0.1308      0.005    -26.891      0.000        -0.140    -0.121
age                -0.0110      0.002     -5.937      0.000        -0.015    -0.007
yrs_married         0.0201      0.002     10.327      0.000         0.016     0.024
children           -0.0008      0.006     -0.134      0.893        -0.012     0.011
religious          -0.0685      0.006    -11.119      0.000        -0.081    -0.056


In [67]:
# Test on the 1000th respondent.

respondent1000 = dta.ix[1000]
print(respondent1000)

rate_marriage       4.000000
age                37.000000
yrs_married        23.000000
children            3.000000
religious           3.000000
educ               12.000000
occupation          3.000000
occupation_husb     4.000000
affairs             0.521739
affair              1.000000
Name: 1000, dtype: float64


In [68]:
resp = dict(zip(range(1,9), respondent1000[["occupation", "educ",
                                            "occupation_husb", "rate_marriage",
                                            "age", "yrs_married", "children",
                                            "religious"]].tolist()))
resp.update({0 : 1})
print(resp)

{0: 1, 1: 3.0, 2: 12.0, 3: 4.0, 4: 4.0, 5: 37.0, 6: 23.0, 7: 3.0, 8: 3.0}


Here, we added a 0th dimension for the bias, with weight 1. 

In [69]:
affair_mod.predict(respondent1000)

array([ 0.51878156])

In [70]:
affair_mod.fittedvalues[1000]

0.075161592850559344

In [71]:
affair_mod.model.cdf(affair_mod.fittedvalues[1000])

0.51878155721214614

## Round 2

In [73]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
support = np.linspace(-6, 6, 1000)
ax.plot(support, stats.logistic.cdf(support), 'r-', label='Logistic')
ax.plot(support, stats.norm.cdf(support), label='Probit')
ax.legend();